In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:

pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [5]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
import json
with open('mpd_slice_0_999.json') as json_file:
    data = json.load(json_file)

In [7]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_conc(artists):
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_conc = unique_artists / total_appearances
    return artist_conc

In [8]:
MS_IN_DAY = 86400
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = []
    dates = []
    tracks = sp.tracks(uris)['tracks']
    for track in tracks:
        if track is None:
            print(track)
        else:
            for artist in track['artists']:
                artists.append(artist['name'])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates.append(ms_date / MS_IN_DAY)
            
    return artists, dates

In [9]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(playlist_uris):
    remainder = len(playlist_uris) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(playlist_uris):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    remainder = len(playlist_uris) % 50
    artists, dates = [], []
    if remainder > 0:
        artists, dates = get_artists_and_dates(playlist_uris[:remainder])
    while remainder < len(playlist_uris):
        next_artists, next_dates = get_artists_and_dates(playlist_uris[remainder:remainder+50])
        artists += next_artists
        dates += next_dates
        remainder += 50
    return all_features, artists, dates

In [10]:
def deduplicate_uris(playlist_uris, used_tracks_by_uri):
    #print(playlist_uris)
    #print(np.array(list(used_tracks_by_uri.keys())))
    #duplicates = np.intersect1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    new_uris = np.setdiff1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    duplicates = set(playlist_uris).intersection(used_tracks_by_uri)
    print('len(playlist_uris) =', len(playlist_uris))
    print('len(duplicates) =', len(duplicates))
    print('len(new_uris) =', len(new_uris))
    return duplicates, new_uris
    #return [], playlist_uris

In [11]:
# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features(tracks, used_tracks_by_uri):

    tracks = pd.DataFrame(tracks)
    duplicates, new_uris = deduplicate_uris(tracks['track_uri'], used_tracks_by_uri) # deduplicate here

    all_features, artists, dates = get_playlist_audio_features_batch(new_uris)
    # updated list of already used tracks
    for row in all_features:
        used_tracks_by_uri[row['uri']] = row
    # add duplicates to this playlist for aggregate metric calculation
    print(len(duplicates))
    for uri in duplicates:
        all_features.append(used_tracks_by_uri[uri])
    all_features = pd.DataFrame(all_features)
    #print(all_features)

    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)
    aggregate_features['date'] = np.mean(dates)
    aggregate_features['date_var'] = np.var(dates)
    aggregate_features['artist_conc'] = get_artist_conc(artists)
    return aggregate_features

In [12]:
# build the aggregate dataset!
def build_dataset(playlists):
    aggregate_dataset = []
    used_tracks_by_uri = {}
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features(playlist['tracks'], used_tracks_by_uri))
    return aggregate_dataset

In [13]:
dataset = pd.DataFrame(build_dataset(data['playlists'][:6]))


AttributeError: 'list' object has no attribute 'intersection'

In [ ]:
dataset.to_csv(r'csvdata_nodup_10.csv')
print(dataset)

   danceability  danceability_var    energy  energy_var  loudness  \
0      0.668804          0.020467  0.777961    0.016150 -4.908451   
1      0.492382          0.018813  0.695923    0.043638 -8.107974   
2      0.671063          0.011819  0.692953    0.030439 -4.875594   
3      0.514429          0.030802  0.620902    0.058971 -9.618754   
4      0.576235          0.028260  0.650418    0.040555 -7.634529   
5      0.679893          0.022163  0.711640    0.027721 -5.922160   

   loudness_var  speechiness  speechiness_var  acousticness  acousticness_var  \
0      1.740075     0.104486         0.007470      0.085282          0.014344   
1      6.896997     0.091010         0.013898      0.162227          0.044290   
2      4.740519     0.096425         0.006835      0.269100          0.059733   
3     18.742903     0.067000         0.004607      0.273514          0.103539   
4      6.233682     0.041218         0.000193      0.177189          0.054154   
5      5.979475     0.078753  